# Исследование надежности заемщиков.

# 1. Обзор данных

In [1]:
import pandas as pd


In [2]:
bank_clients = pd.read_csv('/datasets/data.csv')
bank_clients.to_csv('proj.csv', index=False)

In [3]:
bank_clients.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
bank_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB



Всего в таблице 12 столбцов с типами float, int и object. Количество значений в столбцах отличается, это значит, что в столбцах есть пропущенные значения. Некоторые данные представлены в вещественном типе, его нужно изменить на целочисленный для удобства подсчёта.

Причинами появления пропусков могут быть как человеческий фактор, так и ошибки системы или процесса выгрузки. В данном случае можно отследить закономерность - в строках с пропущенными значениями days_employed также пропущены значения total_income. Вероятность пропуска зависит от других значений в наборе данных, но не от значений собственного столбца. Эти пропуски случайны.



# 2.   Заполнение пропусков. Проверка данных на аномалии

В столбце days_employed найдены пропуски и артефакты - в некоторых случаях стаж отрицательный, в остальных он неправдоподобно большой. Скорее всего, база данных была ошибочно заполнена, либо данные были неправильно извлечены. Избавимся от этого столбца для удобства работы - для дальнейших расчетов он не понадобится.

Пропуски также найдены в столбце total_income. Этот столбец нам понадобится, поэтому заменим пропущенные значение средним арифметическим или медианой, чтобы не потерять другие данные. Так как речь идёт о доходе, для заполнения лучше использовать медиану, чтобы сильно выделяющиеся значения не повлияли на общий результат. Медиану рассчитаем отдельно для каждого типа дохода.

In [5]:
bank_clients.drop(['days_employed'], axis='columns', inplace=True)

In [6]:
bank_clients['total_income'] = bank_clients['total_income'].fillna(-1)

In [7]:
def fill_income(row):
    total_income = row['total_income']
    income_type = row['income_type']
    if total_income == -1:      
        return income_type_to_median[income_type]
    return total_income
 
income_type_to_median = bank_clients.groupby("income_type").median()["total_income"].to_dict()    
       
bank_clients['total_income'] = bank_clients.apply(fill_income, axis=1)

bank_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   dob_years         21525 non-null  int64  
 2   education         21525 non-null  object 
 3   education_id      21525 non-null  int64  
 4   family_status     21525 non-null  object 
 5   family_status_id  21525 non-null  int64  
 6   gender            21525 non-null  object 
 7   income_type       21525 non-null  object 
 8   debt              21525 non-null  int64  
 9   total_income      21525 non-null  float64
 10  purpose           21525 non-null  object 
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


Проверим столбец dob_years.

In [8]:
bank_clients['dob_years'].value_counts().tail(15)

66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце есть нулевые значения, заменим их на среднее арифметическое - в случае возраста такой метод допустим.

In [9]:
bank_clients.loc[bank_clients['dob_years'] == 0] = int(bank_clients['dob_years'].mean())
bank_clients.loc[bank_clients['dob_years'] == 0].count().sum()

0

Проверим столбец children на аномалии.

In [10]:
bank_clients['children'].value_counts()

 0     14080
 1      4802
 2      2042
 3       328
 43      101
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце встречаются отрицательные и аномально большие значения. Доля неверных значений - меньше одного процента, на результат исследования они не повлияют. Избавимся от этих строчек для правильности подсчетов.

In [11]:
bank_clients = bank_clients.loc[bank_clients['children'] >= 0]
bank_clients = bank_clients.loc[bank_clients['children'] < 20]
bank_clients['children'].value_counts()

0    14080
1     4802
2     2042
3      328
4       41
5        9
Name: children, dtype: int64

Вывод
В данных найдены пропуски и ошибки. Мы избавились от ошибочных данных, которые нам не понадобятся, заполнили NaN значения и изменили неправильные в данных, которые нужны.

# 3.  Изменение типов данных.

In [12]:
bank_clients['total_income'] = bank_clients['total_income'].astype(int)
bank_clients['total_income'].dtype

dtype('int64')

Все данные представлены в довольно удобном формате. Месячную зарплату и стаж в днях перевели в 'int', чтобы видеть целочисленные значения. В остальном типа данных везде выглядят хорошо.

# 4. Удаление дубликатов.

Значения в столбце education отличаются регистром. Применим к столбцу метод value_counts() для проверки, а после приведем значения к одному регистру.

In [13]:
bank_clients['education'].value_counts()

среднее                13609
высшее                  4666
СРЕДНЕЕ                  764
Среднее                  700
неоконченное высшее      663
ВЫСШЕЕ                   270
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [14]:
bank_clients['education'] = bank_clients['education'].str.lower()
bank_clients['education'].value_counts()

среднее                15073
высшее                  5202
неоконченное высшее      739
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь установим наличие полных дубликатов и избавимся от них.

In [15]:
bank_clients.duplicated().sum()

71

In [16]:
bank_clients = bank_clients.drop_duplicates().reset_index(drop=True)
bank_clients.duplicated().sum()

0

Дубликаты в данных могли появиться в результате сбоев или неправильных скриптов. Мы использовали метод value_counts() для ознакомления со всеми значениями столбца education, привели их к одному регистру, а после удалили полные дубликаты используя метод drop_duplicates().

# 5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [17]:
education_dict = bank_clients[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
display(education_dict.head(10))

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


Выделили словарь для education,удалили дубликаты из словаря и проверили содержимое

Аналогичным образом поступим для family_status.

In [18]:
family_status_dict = bank_clients[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
display(family_status_dict.head(10))

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Так как словари мы сохранили, id проставлены, удалим столбцы education и family_status из основной таблицы

In [19]:
bank_clients = bank_clients.drop('education', axis=1)
bank_clients = bank_clients.drop('family_status', axis=1)
bank_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21231 entries, 0 to 21230
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21231 non-null  int64 
 1   dob_years         21231 non-null  int64 
 2   education_id      21231 non-null  int64 
 3   family_status_id  21231 non-null  int64 
 4   gender            21231 non-null  object
 5   income_type       21231 non-null  object
 6   debt              21231 non-null  int64 
 7   total_income      21231 non-null  int64 
 8   purpose           21231 non-null  object
dtypes: int64(6), object(3)
memory usage: 1.5+ MB


# 6. Категоризация дохода.

In [20]:
def category_definition(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        if 30001 <= income <= 50000:
            return 'D'
        if 50001 <= income <= 200000:
            return 'C'
        if 200001 <= income <= 1000000:
            return 'B'
        if income >= 1000001:
            return 'A'
    except:
        print('Ошибка')


In [21]:
bank_clients['total_income_category'] = bank_clients['total_income'].apply(category_definition)
display(bank_clients['total_income_category'].value_counts())


C    15849
B     4988
D      347
A       25
E       22
Name: total_income_category, dtype: int64

Применяем функцию category_definition к каждой строке датафрейма, 
результат записывается в новый столбец total_income_category.Проверяем с помощью value_counts()

# 7. Категоризация целей кредита.

In [22]:
display(bank_clients['purpose'].head(15))

0                         покупка жилья
1               приобретение автомобиля
2                         покупка жилья
3            дополнительное образование
4                       сыграть свадьбу
5                         покупка жилья
6                     операции с жильем
7                           образование
8                 на проведение свадьбы
9               покупка жилья для семьи
10                 покупка недвижимости
11    покупка коммерческой недвижимости
12                      сыграть свадьбу
13              приобретение автомобиля
14           покупка жилой недвижимости
Name: purpose, dtype: object

Изучим данные в столбце purpose и определим, какие подстроки помогут правильно определить категорию.

In [23]:

def change_purpose(row):
    purpose = row['purpose']
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif 'жил' in purpose:
        return 'операции с недвижимостью'
    elif 'недвиж' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образ' in purpose:
        return 'получение образования'
    else:
        return 'не входит ни в одну из котегорий'
bank_clients['purpose_category'] = bank_clients.apply(change_purpose,axis=1)
print(bank_clients['purpose_category'].head(15))

0     операции с недвижимостью
1       операции с автомобилем
2     операции с недвижимостью
3        получение образования
4           проведение свадьбы
5     операции с недвижимостью
6     операции с недвижимостью
7        получение образования
8           проведение свадьбы
9     операции с недвижимостью
10    операции с недвижимостью
11    операции с недвижимостью
12          проведение свадьбы
13      операции с автомобилем
14    операции с недвижимостью
Name: purpose_category, dtype: object


Создаём функцию, которая на основании данных из столбца purpose сформирует новый столбец purpose_category, в который войдут следующие категории: 
'операции с автомобилем',
'операции с недвижимостью',
'проведение свадьбы',
'получение образования'. Методом apply применяем функцию ко всему датафрейму и добавляем в новый столбец датафрейма.

# Ответы на вопросы.

# Вопрос 1:
Есть ли зависимость между количеством детей и возвратом кредита в срок?


In [ ]:
#функция, что посчитает нам отношение в процентах
def make_proportion(pdSerises):
    return str(round((pdSerises.sum() / pdSerises.count()) * 100, 2)) + '%'

# построим сводную таблицу для ответа на вопрос
data_pivot = bank_clients.pivot_table(index=['children'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем,по возрастанию долю % для удобства
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
display(data_pivot)

,sum,count,make_proportion
,debt,debt,debt
children,,,
5,0,9,0.0%
0,1058,14022,7.55%
3,27,328,8.23%
1,441,4792,9.2%
2,194,2039,9.51%
4,4,41,9.76%


# Вывод 
Доля заёмщиков с детьми - выше, чем бездетных.Но у должников с пятью детьми просрочки задолжности не наблюдается.

# Вопрос 2:
Есть ли зависимость между семейным положением и возвратом кредита в срок?



In [25]:
debtors_family_status = bank_clients[['family_status_id', 'debt']]
debtors_family_status_grouped = debtors_family_status.groupby('family_status_id').count()
debtors_family_status_grouped['number'] = debtors_family_status.loc[debtors_family_status['debt'] == 1].groupby('family_status_id').sum()
debtors_family_status_grouped['mean, %'] = debtors_family_status_grouped['number'] / debtors_family_status_grouped['debt'] * 100
debtors_family_status_grouped.sort_values('mean, %', ascending=False)

,debt,number,"mean, %"
family_status_id,,,
4,2780,272,9.784173
1,4113,383,9.311938
0,12213,923,7.557521
3,1179,84,7.124682
2,946,62,6.553911


<div class="alert alert-info"> <b> Добавлю вариант с применением фурнкции из предыдущего вопроса и использованием сводных таблиц.

In [26]:
# Так как  мы до этого удаляли из основных данных текстовые значения family_status, тут необходимо их вернуть
df_with_family_status = bank_clients.merge(family_status_dict, on='family_status_id', how='left')

# построим сводную таблицу для ответа на вопрос
data_pivot = df_with_family_status.pivot_table(index=['family_status'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем для удобства
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
display(data_pivot)

,sum,count,make_proportion
,debt,debt,debt
family_status,,,
вдовец / вдова,62,946,6.55%
в разводе,84,1179,7.12%
женат / замужем,923,12213,7.56%
гражданский брак,383,4113,9.31%
Не женат / не замужем,272,2780,9.78%


# Вывод
Результат исследования показывает, что в группе клиентов со статусом не женат / не замужем доля должников больше всего - почти 9,8%. Меньше всего должников среди вдовцов - 6,5%.



# Вопрос 3:
Есть ли зависимость между уровнем дохода и возвратом кредита в срок?


In [27]:
# построим сводную таблицу для ответа на вопрос
data_pivot = bank_clients.pivot_table(index=['total_income_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем для удобства
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
display(data_pivot)

,sum,count,make_proportion
,debt,debt,debt
total_income_category,,,
D,21,347,6.05%
B,353,4988,7.08%
A,2,25,8.0%
C,1346,15849,8.49%
E,2,22,9.09%


# Вывод
В результате подсчета в этих категорих видим, что меньше всего просрочки у людей категории 'D',но их небольшое количество.Доля просрочки кредита более высокая у людей категории 'C' и 'E', но опять же люди с самым меньшим доходом категории 'E' реже берут кредиты.  

# Вопрос 4:
Как разные цели кредита влияют на его возврат в срок?


In [28]:
# построим сводную таблицу для ответа на вопрос
data_pivot = bank_clients.pivot_table(index=['purpose_category'], values=["debt"], aggfunc=['sum', 'count', make_proportion])
# сортируем для удобства
data_pivot = data_pivot.sort_values(by=('make_proportion', 'debt'))
display(data_pivot)

,sum,count,make_proportion
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,777,10704,7.26%
проведение свадьбы,181,2299,7.87%
получение образования,369,3970,9.29%
операции с автомобилем,397,4258,9.32%


# Вывод
В результате группирования по категории цели кредита видим, что количество должников в категориях "операции с автомобилем" и "получение образования" преобладает.

# Общий вывод:

В ходе работы искали зависимость между:

наличием детей и возвратом кредита в срок - доля должников среди заемщиков с детьми выше</b>;

семейным положением и возвратом кредита в срок - зависимость есть, чаще всего становятся должниками люди с семейным положением не женат / не замужем</b>;

уровнем дохода и возвратом кредита в срок - выраженная зависимость между уровнем дохода и возвратом кредита в срок не установлена</b>;

целью кредита и возвратом кредита в срок - зависимость есть, большая доля должников взяла кредит на автомобиль или образование</b>.

Как результат, выявили зависимость между потециальным долгом и семейным положением, наличием детей и целью кредита. Наличие зависимости между уровнем дохода и возвратом кредита в срок не подтверждено.